# ChIP-seq analysis #

In [ ]:
# Functions used in this notebook
read.bed <- function(file.bed, col.names = replicate(2, c("chrom", "start", "end", "name", "score"))) {
    bed <- read.delim(file.bed, header = FALSE, sep = "\t",
                      col.names = col.names)
    # Drop empty columns, with all NAs
    col.na <- sapply(bed, function(x) all(is.na(x)))
    col.keep <- names(col.na)[!col.na]
    bed[,col.keep]
}

write.bed <- function(df, file.bed) {
    write.table(df, file = file.bed, quote = FALSE, sep = "\t",
                row.names = FALSE, col.names = FALSE)
}
                     
counts2fpkm <- function(counts, length.eff) {
    # See https://archive.is/V0bgu
    exp(log(counts) + log(1e9) + log(length.eff) - log(colSums(counts)))
}

## 1.1 Pipeline script

View [source](chipseq/Makefile)

## 1.2 Summary table ##

In [ ]:
reads.summary <- read.csv('chipseq/summary_reads.csv')
reads.summary

## 1.3 High confidence peaks ##

In [ ]:
file.highconf <- "chipseq/ChIPseq.high_confidence_peaks.bed"
# By default, `bedtools intersect` will require 50% overlap between the regions.
system(paste("bedtools intersect -wo",
             "-a chipseq/ChIPseq_1.chr15_peaks.bed",
             "-b chipseq/ChIPseq_2.chr15_peaks.bed",
             ">", file.highconf))
col.names.each <- c("chrom", "start", "end", "name", "score")
col.names <- c(paste(col.names.each,
                     rep(c("1", "2"), each = length(col.names.each)),
                     sep = "."), "overlap")
peaks.highconf <- read.bed(file.highconf, col.names = col.names)
nrow(peaks.highconf); head(peaks.highconf)

## 1.4 Compare replicates ##

In [ ]:
cor(peaks.highconf$score.1, peaks.highconf$score.2)

In [ ]:
# Recalculate reads for all samples based on exact overlapping range
peaks.exact.overlap.ranges <- data.frame(chrom = peaks.highconf$chrom.1,
                                         start = with(peaks.highconf, pmax(start.1, start.2)),
                                         end = with(peaks.highconf, pmin(end.1, end.2)))
file.exact.overlap.ranges <- "chipseq/peak_overlaps.bed"
write.bed(peaks.exact.overlap.ranges, file.exact.overlap.ranges)
# Use bedtools multicov to get the reads for all datasets
files.bam <- Sys.glob("chipseq/*.chr15.bam")
# Set the column name to the filenames, but without the prefix and file extension
col.names <- unlist(lapply(files.bam, gsub,
                           pattern = "^chipseq/(.*)_(.*)[.]chr15[.]bam$",
                           replacement = "\\2.\\1"))
system(paste("cd chipseq && ls *.chr15.bam | xargs -P $(nproc) -n1 samtools index"))
file.exact.overlap <- "chipseq/peak_overlaps_all.bed"
system(paste("bedtools multicov",
             "-bams chipseq/*.chr15.bam",
             "-bed", file.exact.overlap.ranges,
             ">", file.exact.overlap))
peaks.exact.overlap <- read.bed(file.exact.overlap,
                                col.names = c("chrom", "start", "stop", col.names))
head(peaks.exact.overlap)

In [ ]:
library(ggplot2)
ggplot(peaks[,c("treat.ChIPseq_1", "treat.ChIPseq_2")], aes())

In [ ]:
file.untr <- "chipseq/Input.chr15_peaks.bed"
peaks.untr <- read.bed(file.untr)
nrow(peaks.untr); head(peaks.untr)

## 1.5 Compare distribution of treated and untreated binding sites ##

## 1.6 Compare TF binding between treatments ##

## 1.7 Identify transcription factor binding motif

---

# RNA-seq analysis #

## Pipeline script

View [source](rnaseq/Makefile)

## 2.2 Comparison of replicates

In [ ]:
# Read in the HTseq output as a data.frame
files.htseq <- Sys.glob("rnaseq/*.htseq")
counts <- do.call(cbind.data.frame,
                  lapply(files.htseq, read.delim, header = FALSE,
                         sep = "\t", row.names = 1,
                         col.names = c("gene.id", "counts")))
# Set the column name to the filenames, but without the prefix and file extension
col.names <- unlist(lapply(files.htseq, gsub,
                           pattern = "^rnaseq/RNAseq_(.*)_(.*)[.]htseq$",
                           replacement = "\\2.\\1"))
names(counts) <- col.names
counts <- head(counts, -5) # Drop the last 5 rows since they have summary statistics instead of gene counts
nrow(counts); head(counts)

In [ ]:
file.genes <- "/archive/MCB5429/annotations/hs/Beds/hg19_gencode_ENSG_geneID.bed"
genes <- read.bed(file.genes)
genes$length <- genes$end - genes$start
## Prepare to merge data.frames
counts$gene.id <- rownames(counts)
rownames(counts) <- NULL
## Remove .NN suffix to match gene.id
counts$gene.id <- sub("\\..*", "", counts$gene.id)
names(genes)[names(genes) == "name"] <- "gene.id"
merged <- merge(counts, genes)
## FPKM calculation per https://archive.is/V0bgu
fpkm <- counts2fpkm(merged[,col.names], merged$length)
nrow(fpkm); head(fpkm)

In [ ]:
library(ggplot2)
cor.treat <- cor(fpkm$treat.1, fpkm$treat.2)
ggplot(log2(fpkm), aes(treat.1, treat.2)) +
    geom_point() + geom_smooth() +
    annotate("text", label = sprintf("Correlation = %f", cor.treat),
             x = 30, y = 15)

In [ ]:
cor.untr <- cor(fpkm$untr.1, fpkm$untr.2)
ggplot(log2(fpkm), aes(untr.1, untr.2)) +
    geom_point() + geom_smooth() +
    annotate("text", label = sprintf("Correlation = %f", cor.untr),
             x = 30, y = 15)

## 2.3 Differential Gene Expression

Determined all 4 datasets are single ended and not paired end using
RSeQC per https://www.biostars.org/p/66627/#134380:

```sh
pip install --user RSeQC
ls *.sorted.sam | xargs -n1 ~/.local/bin/infer_experiment.py \
   -r /archive/MCB5429/annotations/hs/Beds/hg19_gencode_ENSG_geneID.bed -i
```

```
...
This is SingleEnd Data
Fraction of reads failed to determine: 0.0150
Fraction of reads explained by "++,--": 0.4927
Fraction of reads explained by "+-,-+": 0.4923
...
```

In [ ]:
suppressMessages(library(DESeq2))
## Create DESeq2 object using DESeqDataFromMatrix
## 
## Generate countData
countData <- merged[col.names]
rownames(countData) <- merged$gene.id
## Generate colData
conditioner <- function(s) if (grepl("untr", s)) "untreated" else "treated"
condition <- unlist(lapply(col.names, conditioner))
type <- replicate(length(condition), "single-read")
colData <- cbind.data.frame(condition, type)
rownames(colData) <- col.names
## Combine into DESeq2 object
dds <- DESeqDataSetFromMatrix(countData = countData,
                              colData = colData,
                              design = ~ condition)
dds$condition <- relevel(dds$condition, ref = "untreated")
dds <- DESeq(dds)
dds

In [ ]:
res <- results(dds)
plotMA(res, ylim = c(-1.5, 2))

## 2.4 Parse out regulated genes

In [ ]:
res.df <- as.data.frame(res)
## Get significant rows with pvalue < 0.05
res.df <- na.omit(res.df[res.df$pvalue < 0.05,])
## Regulated gene names
res.df$reg <- as.factor(ifelse(res.df$log2FoldChange > 0, "up", "down"))
reg <- rownames(res.df)
head(res.df)

In [ ]:
upReg <- rownames(res.df[res.df$reg == "up",])
downReg <- rownames(res.df[res.df$reg == "down",])
write.csv(res.df, file = "rnaseq/reg.csv", quote = FALSE)
## Write bed file of regulated genes
reg.bed <- subset(read.bed(file.genes), name %in% reg)
nrow(reg.bed); head(reg.bed)

In [ ]:
write.bed(reg.bed, "rnaseq/reg.bed")

## 2.5 Distances between regulated gene TSS and ChIP-seq peaks

In [ ]:
file.chip.peaks <- "/archive/MCB5429/Final_data/ChIPseq/MACSout/ChIP_overlap_treat_peaks_allChr.bed"
system(paste("bedtools closest -d -a rnaseq/reg.bed -b", file.chip.peaks,
             "> rnaseq/reg.dist"))
col.names.dist <- c("chrom", "start", "end", "gene.id", "score",
                    "strand", "chrom.peak", "start.peak", "stop.peak",
                    "peak.id", "score.peak", "dist")
dist <- read.delim("rnaseq/reg.dist", header = FALSE, sep = "\t",
                   col.names = col.names.dist)
nrow(dist); head(dist)

In [ ]:
res.df$gene.id <- rownames(res.df)
rownames(res.df) <- NULL
merged <- merge(dist, res.df, by = "gene.id")
ggplot(merged, aes(log2(dist), color = reg)) + stat_ecdf()

## 2.6 Screenshot from Browser

In [ ]:
# Select interesting genes to view in the genome browser
within.1k <- merged[merged$dist <= 1000,]
sorted <- within.1k[with(within.1k,
                         order(dist, -abs(log2FoldChange), score.peak)),]
head(sorted)

Interestingly, the top hits on chromosome 19 and 17 only appear in
untr.2 and none of the other samples, so it seems like an artifact.
Therefore, I look at the next highest hits on chr3 instead.